In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

pipeline_config = 'path_to_retinanet_config_file.config'
configs = config_util.create_configs_from_pipeline_proto(pipeline_config)
model = tf.saved_model.load(configs['model'])


In [ ]:
import numpy as np
import tensorflow as tf
import os
import pathlib
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import image_utils

# Load the model
PATH_TO_SAVED_MODEL = 'path/to/retinanet_model/saved_model'
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# Load label map
PATH_TO_LABELS = 'path/to/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load image
image_path = 'path_to_image.jpg'
image_np = np.array(image_utils.load_image_into_numpy_array(image_path))

# Run detection
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis,...]

# Perform the detection
output_dict = detect_fn(input_tensor)

# All outputs are float32 numpy arrays, so convert to appropriate types
num_detections = int(output_dict.pop('num_detections'))
output_dict['detection_classes'] = np.array(output_dict['detection_classes'][0]).astype(np.int64)
output_dict['detection_boxes'] = np.array(output_dict['detection_boxes'][0])
output_dict['detection_scores'] = np.array(output_dict['detection_scores'][0])

# Visualize the results
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks', None),
    use_normalized_coordinates=True,
    line_thickness=8)

# Show the image with detection boxes
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
plt.imshow(image_np)
plt.show()
